## Комментарий наставника
Привет, Евгений! Меня зовут Александр и я буду проверять твой проект. Спасибо за твою работу:) Далее по ходу работы я оставлю свои комментарии и предложения. Постарайся их учесть в этом и дальнейших проектах. Комментарии ты можешь найти в текстовой ячейке с заголовком «Комментарий наставника» (как здесь) либо в ячейках с кодом в следующем виде: «#Комментарий наставника: <сам комментарий>». \
Часть комментариев может быть выделена цветом: \
<span style="color:green">Зелёный цвет символизирует, что всё отлично</span> \
<span style="color:orange">Оранжевый цвет символизирует рекомендации</span> \
<span style="color:red">Красный цвет символизирует недочёты</span>

<font color=violet>  
    
#### Александр, привет. Спасибо за ревью, обьективную критику и полезные советы, я буду писать фиолетовым)

<span style="color:green">Рад был помочь!:)</span>

<font color=darkblue> 

## Описание задачи
___
    
### Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

## Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

__Вызываю библиотеки__

In [1]:
import pandas as pd
import itertools as itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats as st
from matplotlib.gridspec import GridSpec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_curve, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier

<font color=darkblue> 
# Подготовка данных

__Открываю датасет__

In [2]:
behavior = pd.read_csv('https:/')

In [3]:
behavior

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


__Проверю типы__

In [4]:
behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Вывод

- Пропусков нет.
- Типы данных приведены.



## Комментарий наставника
<span style="color:green">Хорошо, данные получены. Идём далее.</span>

<font color=darkblue> 
# Подготовка дата сетов

__Что делаю__
- Разделю исходные данные на обучающую, валидационную и тестовую выборки.

__Получу обучающую вборку в 60 процентов__

In [5]:
behavior_train, behavior_test_valid = train_test_split(behavior, test_size=0.4, random_state=12345)

__Получу тестовую и валидационную выборку в 20 процентов каждая__

In [6]:
behavior_test, behavior_valid = train_test_split(behavior_test_valid, test_size=0.5, random_state=12345)

__Проверю параметры выборок__

In [7]:
print('Dataset:', len(behavior),
      'Осовная выборка:', len(behavior_train),
      'Тестовая выборка:', len(behavior_test), 
      'Валидационная выборка:', len(behavior_valid))

Dataset: 3214 Осовная выборка: 1928 Тестовая выборка: 643 Валидационная выборка: 643


### Вывод

- Датасет разделен на три выборки в пропорции 60/20/20.



## Комментарий наставника
<span style="color:green">Прекрасно!</span> \
<span style="color:orange">Отделяй признаки для обучения от целевых значений и сохраняй их в отдельные переменные, которые потом сможешь использовать для обучения моделей и получения предсказаний. Это можно сделать таким образом:</span>

<font color=violet>     
#### Разделил признаки и исправил все дальше по проекту)

<span style="color:green">UPD 07.05.2020 Класс! Согласись, что так намного удобнее работать с данными.</span>

In [8]:
features_train = behavior_train.drop(['is_ultra'], axis=1)
target_train = behavior_train['is_ultra']

In [9]:
features_valid = behavior_valid.drop(['is_ultra'], axis=1)
target_valid = behavior_valid['is_ultra']

In [10]:
features_test = behavior_test.drop(['is_ultra'], axis=1)
target_test = behavior_test['is_ultra']

<font color=darkblue> 
# Построение моделей

## Модель решающего дерева

### Определю параметры перебором

In [11]:
for depth in range(1, 6):
    model_tree = DecisionTreeClassifier(random_state=12345,max_depth=depth) 
    model_tree.fit(features_train, target_train)
    predictions = model_tree.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions))

max_depth = 1 : 0.7356143079315708
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7791601866251944
max_depth = 4 : 0.7744945567651633
max_depth = 5 : 0.7838258164852255


## Комментарий наставника
<span style="color:red">Когда мы подбираем гиперпараметры для моделей, мы используем валидационную выборку для оценки. Тестовую выборку на данном этапе трогать не нужно, она нам нужна для получения финальной оценки нашей лучшей модели (или нескольких лучших), выбранной среди всех по результатам оценки по валидационной выборке. То же самое касается и последующих твоих моделей.</span> \
<span style="color:orange">Здесь уже как раз можешь использовать готовые переменные, содержащие признаки для обучения и целевые значения, которые я создал выше. Тогда не придётся постоянно отделять признаки для обучения от целевых значений каждый раз, когда мы обучаем модель.</span>

<font color=violet>     
#### Чет да, понаперепутал все) Поправил

In [12]:
for maxi in range(1, 5):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=5, max_features = maxi) 
    model_tree.fit(features_train, target_train)
    predictions = model_tree.predict(features_valid)
    print("max_features =", maxi, ": ", end='')
    print(accuracy_score(target_valid, predictions))

max_features = 1 : 0.7822706065318819
max_features = 2 : 0.7884914463452566
max_features = 3 : 0.7822706065318819
max_features = 4 : 0.7838258164852255


## Комментарий наставника
<span style="color:orange">UPD 07.05.2020 Ты подбираешь ограничения по количеству признаков, а выводишь это как глубину дерева.</span>

<font color=violet>     
#### UPD 07.05 Поправил

#### Модель решающего дерева с параметром глубины 3

In [13]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=3, max_features = 3).fit(features_train,
target_train)

### Подберу параметры методом Grid

#### Возьму обьедененные выборки (тренировочную и валидационную)

In [14]:
grid = pd.concat([behavior_train, behavior_valid], ignore_index=True)
features_grid = grid.drop(['is_ultra'], axis=1)
target_grid = grid['is_ultra']

In [15]:
params_t = {'max_depth':   [x for x in range (1,20)], 'max_features':[x for x in range (1,5)],}

#### Применю кросс-валидацию

In [16]:
grid_tree = GridSearchCV(DecisionTreeClassifier(), params_t, cv = 5)

In [17]:
grid_tree.fit(features_grid, target_grid)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17,

In [18]:
Q = grid_tree.best_params_

In [19]:
Q

{'max_depth': 9, 'max_features': 4}

### На основании лучших параметров обучу модель

#### Применю параметры метода Grid

In [20]:
model_tree_grid = DecisionTreeClassifier(random_state=12345, max_depth= Q['max_depth'], 
                                         max_features = Q['max_features']).fit(features_train, target_train)

In [21]:
model_tree_grid_predict = model_tree.predict(features_valid)

In [22]:
accuracy_score(target_valid, model_tree_grid_predict)

0.7791601866251944

## Комментарий наставника
<span style="color:red">UPD 07.05.2020 Не забывай оценивать модель по валидационной выборке. Уже дополнительно к ней ты можешь выводить оценку и по тренировочной выборке.</span>

<font color=violet>     
#### UPD 07.05 Поправил в кросс валидации.  В подборе я оцениваю прям в цикле.

#### Оценка лучшей модели

In [23]:
grid_tree.best_score_

0.8028004667444574

### Вывод: на соновании кросс-валидации, и исходя из принципа необходимой достаточности, принимаю для дерева параметры
### max_depth = 3, max_features = 4

In [24]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=3, max_features = 4).fit(features_train,
target_train)

## Вывод

- Благодаря Grid, удалось поднять accurancy до 80

## Комментарий наставника
<span style="color:green">Ещё и кросс-валидация есть, молодец!:)</span> \
<span style="color:orange">Если используешь кросс-валидацию, то можешь использовать для подбора параметров комбинированную выборку (объединить тренировочную и валидационную выборки в одну), так как на каждой итерации кросс-валидации наша выборка сама разделяется на 2 непересекающиеся части: тренировочную и валидационную. Плюс подхода в том, что у нас будет больше данных для обучения модели. И оценку лучшей модели, кстати, ты тоже можешь получить без повторного обучения модели. Делается это следующей командой:</span>

<font color=violet>     
#### Объеденил тренировочную и валидационную выборку, и привел лучший скор после кросс-валидации, но он почему-то оказался ниже чем аккуранси

## Модель случайного леса

### Определю параметры перебором

In [25]:
for n_estimators in range(1,10):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=n_estimators) 
    model_forest.fit(features_train, target_train)
    predictions_forest = model_forest.predict(features_valid)
    print("max_trees =", n_estimators, ": ", end='')
    print(accuracy_score(target_valid, predictions_forest))


max_trees = 1 : 0.7402799377916018
max_trees = 2 : 0.7589424572317263
max_trees = 3 : 0.7573872472783826
max_trees = 4 : 0.7729393468118196
max_trees = 5 : 0.7667185069984448
max_trees = 6 : 0.7791601866251944
max_trees = 7 : 0.7807153965785381
max_trees = 8 : 0.7869362363919129
max_trees = 9 : 0.7838258164852255


## Комментарий наставника
<span style="color:red">UPD 07.05.2020 Здесь у тебя опять появилась тестовая выборка :(</span>

<font color=violet>     
#### UPD 07.05 Поправил, обещаю запомнить это)

In [26]:
for depth in range(2, 20, 2):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=6, max_depth = depth) 
    model_forest.fit(features_train, target_train)
    predictions_forest = model_forest.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_forest))

max_depth = 2 : 0.76049766718507
max_depth = 4 : 0.7853810264385692
max_depth = 6 : 0.7869362363919129
max_depth = 8 : 0.8055987558320373
max_depth = 10 : 0.7978227060653188
max_depth = 12 : 0.7884914463452566
max_depth = 14 : 0.7931570762052877
max_depth = 16 : 0.7869362363919129
max_depth = 18 : 0.776049766718507


#### Модель случайного леса с 6 деревьями и глубиной 8

In [27]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators=6, max_depth=8, n_jobs=-1).fit(features_train, target_train)

### Подберу параметры методом Grid

In [28]:
params_rf = {'max_depth':   [x for x in range (1,20)], 
             'n_estimators':[x for x in range (2,40)]
         }

In [29]:
grid_forest = GridSearchCV(RandomForestClassifier(), params_rf, refit = True, cv = 5, n_jobs=-1)

#### Возьму обьедененные выборки (тренировочную и валидационную)

In [30]:
grid_forest.fit(features_grid, target_grid)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [31]:
W = grid_forest.best_params_

In [32]:
W

{'max_depth': 10, 'n_estimators': 38}

### На основании лучших параметров обучу модель

In [33]:
model_forest_grid = RandomForestClassifier(random_state=12345, n_estimators= W['n_estimators'],
                                           max_depth=W['max_depth'], n_jobs=-1).fit(features_train, target_train)

In [34]:
model_forest_grid__predict = model_forest_grid.predict(features_valid)

In [35]:
accuracy_score(target_valid, model_forest_grid__predict)

0.7993779160186625

#### Оценка лучшей модели

In [36]:
grid_forest.best_score_

0.8136911707506806

### Вывод: на соновании кросс-валидации и исходя из принципа необходимой достаточности, принимаю для леса параметры
### n_estimators= 6, max_depth=8

In [37]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators= 6,
                                           max_depth=8, n_jobs=-1).fit(features_train, target_train)

## Модель логистической регрессии

In [38]:
model_logistic = LogisticRegression(random_state=12345).fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### Обучу модель

In [39]:
model_logistic.score(features_train, target_train)

0.7505186721991701

In [40]:
model_logistic_predict = model_logistic.predict(features_valid)

#### Посчитаю аккураcси

In [41]:
accuracy_score(target_valid, model_logistic_predict)

0.7402799377916018

In [42]:
model_logistic_predict = model_logistic.predict(features_test)

## Тестирование моделей

#### Модель решающего дерева

In [43]:
model_tree = model_tree.predict(features_test)

In [44]:
accuracy_score(target_test, model_tree)

0.7853810264385692

#### Модель случайного леса

In [45]:
model_forest = model_forest.predict(features_test)

In [46]:
accuracy_score(target_test, model_forest)

0.7962674961119751

#### Модель логистической регрессии

In [47]:
model_logistic_predict = model_logistic.predict(features_test)

In [48]:
accuracy_score(target_test, model_logistic_predict)

0.7589424572317263

## Вывод

### Самые высокий аккураси показала модель сслучайного леса с гипермараметрами n_estimators= 6, max_depth=8, примем ее как рабочую.

## Комментарий наставника
<span style="color:red">Какую модель выбираем в итоге? Не забывай комментировать свой код и писать больше промежуточных выводов. \
Следи за структурой проекта, пожалуйста. Добавь блок в проект, посвящённый тестированию (получению предсказаний и оценке твоей лучшей модели уже на тестовой выборке).</span>

<font color=violet>     
#### Добавил коментарии по ходу проекта и блок тестирования с выбором модели проекта

<span style="color:green">UPD 07.05.2020 Отлично!</span>

## Проверка на адекватность

In [49]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_valid, target_valid)

0.6842923794712286

In [50]:
dummy_clf.score(features_test, target_test)

0.7060653188180405

## Вывод

- Все модели выше показателей Дамми-модели, можно сделать вывод, что оценку на адекватность прошли все.

## Комментарий наставника
<span style="color:green">Отлично, проверка на адекватность есть.</span> \
<span style="color:orange">Лучше всего было бы «обучить» нашу дамми-модель на тренировочной выборке, а оценить уже по валидационной/тестовой выборке. Так мы бы смоделировали процесс обучения, но твой вариант тоже подходит вполне.</span>

<font color=violet>     
#### Сделал, как ты сказал. Обучил на тренировочных данных и проверил на валидационных и тестовых.

<span style="color:green">UPD 07.05.2020 Прекрасно, нет вопросов.</span>

## Вывод

Самой точной оказалась модель случайного леса, самой неточной - логистической регрессии.


## Итоговый комментарий наставника
<span style="color:red">Отличная работа! Наличие кросс-валидации меня очень порадовало. Тебе осталось только решить проблему с использованием выборок и добавить отдельно блок с тестированием лучшей модели в проект. Если учтёшь мои дополнительные рекомендации в жёлтом цвете, то будет вообще круто. Будем ждать твою работу:) \
UPD 07.05.2020 Обрати внимание, что тестовая выборка у тебя по-прежнему используется при подборе параметров для модели случайного леса. Комментарии я добавил.</span>

<font color=violet>     
####  UPD 07.05.2020 Поправил, постараюсь быть внимательнее

<span style="color:green">UPD 08.05.2020 Прекрасная работа! Ты сделал даже больше, чем требовалось по проекту. Так держать! Поздравляю с зачётом и желаю дальнейших успехов в учёбе:)</span>